# References
- [SFTTrainter Code](https://github.com/huggingface/trl/blob/main/trl/trainer/sft_trainer.py)
- [SFT Trainer Tutorial](https://huggingface.co/docs/trl/en/sft_trainer)
- [gemma finetuing with peft](https://huggingface.co/blog/gemma-peft)
- [lora](https://huggingface.co/docs/peft/main/en/developer_guides/lora#rank-stabilized-lora)


- [TrainingArguments](https://huggingface.co/docs/transformers/v4.38.2/en/main_classes/trainer#transformers.TrainingArguments)
- [model.generate](https://huggingface.co/docs/transformers/v4.38.2/en/model_doc/phi#transformers.PhiForCausalLM.generate)

In [1]:
%%capture
!pip install -qq wandb datasets transformers bitsandbytes peft accelerate trl[peft]

In [2]:
from google.colab import userdata
import wandb
import os

os.environ["WANDB_PROJECT"] = "text-ads-generation"  # name your W&B project
os.environ["WANDB_LOG_MODEL"] = "checkpoint"  # log all model checkpoints
os.environ["WANDB_WATCH"] = "all"  # watch params and gradients

wandb.login(key=userdata.get('WANDB_API_KEY'))

wandb: Currently logged in as: codescv. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [3]:
run_no = 9
run_name = f"gemma-2b-{run_no}"
output_dir = f"outputs-{run_no}"
!rm -rf {output_dir}
!rm -rf wandb
max_seq_length = 384
data_dir = '/content/drive/MyDrive/02 data/text_ads_generation'

In [4]:
from datasets import Dataset
import yaml
import json
import glob

sft_template = """### Human: Generate {count} text ads for the product with the following information.
{information}
### Assistant:
"""

def sft_transform(data_dir, label=True):
  for file in glob.glob(f'{data_dir}/*.jsonl'):
    with open(file) as f:
      for line in f:
        item = json.loads(line)
        ad_headlines = item['ad_headlines']
        answer = '\n'.join(ad_headlines)
        del(item['ad_headlines'])
        information = yaml.dump(item)
        text = sft_template.format(count=len(ad_headlines), information=information)
        if label:
          text += f"{answer}"
        yield {'text': text}

for data in sft_transform(data_dir):
  print(data['text'])
  break

### Human: Generate 10 text ads for the product with the following information.
description: Immerse yourself in the game with this ultra-wide curved gaming monitor.
  With its 4K resolution and HDR support, you'll experience stunning visuals and smooth
  gameplay. The 144Hz refresh rate and 1ms response time ensure blur-free action.
product_category: Electronics -> Computers & Accessories
product_name: Ultra HD 4K Curved Gaming Monitor
selling_point: 4K resolution, HDR support, 144Hz refresh rate, 1ms response time
target_audience: Gamers, video editors, graphic designers

### Assistant:
Immersive 4K Gaming Immersion
Razor-Sharp 144Hz Refresh Rate
Blazing-Fast 1ms Response Time
HDR for Stunning Visuals
Immerse Yourself in the Game
4K Gaming Immersion
HDR Visuals Unleashed
144Hz Blur-Free Action
1ms Response Time Precision
Curved for Cinematic Gaming


In [5]:
import torch
import os
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "google/gemma-2b"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id, token=userdata.get('HF_TOKEN'))
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0}, token=userdata.get('HF_TOKEN'))
# model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map='auto', token=userdata.get('HF_TOKEN'))
model = AutoModelForCausalLM.from_pretrained(model_id, device_map='auto', torch_dtype=torch.bfloat16, token=userdata.get('HF_TOKEN'))
print([(name, param.shape, param.dtype) for name,param in model.named_parameters()])

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

[('model.embed_tokens.weight', torch.Size([256000, 2048]), torch.bfloat16), ('model.layers.0.self_attn.q_proj.weight', torch.Size([2048, 2048]), torch.bfloat16), ('model.layers.0.self_attn.k_proj.weight', torch.Size([256, 2048]), torch.bfloat16), ('model.layers.0.self_attn.v_proj.weight', torch.Size([256, 2048]), torch.bfloat16), ('model.layers.0.self_attn.o_proj.weight', torch.Size([2048, 2048]), torch.bfloat16), ('model.layers.0.mlp.gate_proj.weight', torch.Size([16384, 2048]), torch.bfloat16), ('model.layers.0.mlp.up_proj.weight', torch.Size([16384, 2048]), torch.bfloat16), ('model.layers.0.mlp.down_proj.weight', torch.Size([2048, 16384]), torch.bfloat16), ('model.layers.0.input_layernorm.weight', torch.Size([2048]), torch.bfloat16), ('model.layers.0.post_attention_layernorm.weight', torch.Size([2048]), torch.bfloat16), ('model.layers.1.self_attn.q_proj.weight', torch.Size([2048, 2048]), torch.bfloat16), ('model.layers.1.self_attn.k_proj.weight', torch.Size([256, 2048]), torch.bfloa

In [6]:
import transformers
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM

from peft import LoraConfig
import itertools
import os
import pandas as pd
import wandb

from transformers import TrainerCallback


def sample_generation(step):
  results = []
  for data in itertools.islice(sft_transform(data_dir, label=False), 3):
    text = data['text']
    device = "cuda:0"
    inputs = tokenizer(text, return_tensors="pt").to(device)

    outputs = model.generate(**inputs, max_new_tokens=max_seq_length, do_sample=True, top_k=50, top_p=0.95,
                             pad_token_id= tokenizer.eos_token_id, eos_token_id= tokenizer.eos_token_id, repetition_penalty=1.2)

    sample_result = tokenizer.decode(outputs[0], skip_special_tokens=False)
    results.append({'sample': sample_result, 'step': step})

  return pd.DataFrame(results)


class MyCallBack(TrainerCallback):
  def on_log(self, args, state, control, logs, **kwargs):
    if state.global_step > 50 and state.global_step % 50 == 0:
      sample_result = sample_generation(state.global_step)
      records_table = wandb.Table(dataframe=sample_result)
      print('samples:', sample_result)
      wandb.log({"samples": records_table})



lora_config = LoraConfig(
    r=8,
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
    task_type="CAUSAL_LM",
)

sft_dataset = Dataset.from_generator(sft_transform, gen_kwargs={'data_dir': data_dir, 'label': True})
response_template = "### Assistant:"
collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer)


trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=sft_dataset,
    callbacks=[MyCallBack()],
    args=transformers.TrainingArguments(
        run_name=run_name,
        per_device_train_batch_size=12,
        gradient_accumulation_steps=1,
        warmup_steps=20,
        num_train_epochs=20,
        learning_rate=4e-4,
        # fp16=True,
        bf16=True,
        logging_steps=1,
        output_dir=output_dir,
        optim="paged_adamw_8bit",
        save_strategy="epoch",
        report_to="wandb"
    ),
    max_seq_length=max_seq_length,
    peft_config=lora_config,
    dataset_text_field="text",
    packing=True,
)
trainer.train()

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Step,Training Loss
1,3.651300
2,3.234600
3,3.323500
4,3.164300
5,3.446900
6,3.303100
7,2.941400
8,3.022100
9,3.124100
10,2.763000


samples:                                               sample  step
0  <bos>### Human: Generate 10 text ads for the p...   100
1  <bos>### Human: Generate 10 text ads for the p...   100
2  <bos>### Human: Generate 10 text ads for the p...   100


wandb: Adding directory to artifact (./outputs-9/checkpoint-141)... Done. 0.1s


samples:                                               sample  step
0  <bos>### Human: Generate 10 text ads for the p...   150
1  <bos>### Human: Generate 10 text ads for the p...   150
2  <bos>### Human: Generate 10 text ads for the p...   150
samples:                                               sample  step
0  <bos>### Human: Generate 10 text ads for the p...   200
1  <bos>### Human: Generate 10 text ads for the p...   200
2  <bos>### Human: Generate 10 text ads for the p...   200
samples:                                               sample  step
0  <bos>### Human: Generate 10 text ads for the p...   250
1  <bos>### Human: Generate 10 text ads for the p...   250
2  <bos>### Human: Generate 10 text ads for the p...   250


...

samples:                                               sample  step
0  <bos>### Human: Generate 10 text ads for the p...  2550
1  <bos>### Human: Generate 10 text ads for the p...  2550
2  <bos>### Human: Generate 10 text ads for the p...  2550
samples:                                               sample  step
0  <bos>### Human: Generate 10 text ads for the p...  2600
1  <bos>### Human: Generate 10 text ads for the p...  2600
2  <bos>### Human: Generate 10 text ads for the p...  2600
samples:                                               sample  step
0  <bos>### Human: Generate 10 text ads for the p...  2650
1  <bos>### Human: Generate 10 text ads for the p...  2650
2  <bos>### Human: Generate 10 text ads for the p...  2650


wandb: Adding directory to artifact (./outputs-9/checkpoint-2679)... Done. 0.1s


samples:                                               sample  step
0  <bos>### Human: Generate 10 text ads for the p...  2700
1  <bos>### Human: Generate 10 text ads for the p...  2700
2  <bos>### Human: Generate 10 text ads for the p...  2700
samples:                                               sample  step
0  <bos>### Human: Generate 10 text ads for the p...  2750
1  <bos>### Human: Generate 10 text ads for the p...  2750
2  <bos>### Human: Generate 10 text ads for the p...  2750
samples:                                               sample  step
0  <bos>### Human: Generate 10 text ads for the p...  2800
1  <bos>### Human: Generate 10 text ads for the p...  2800
2  <bos>### Human: Generate 10 text ads for the p...  2800


wandb: Adding directory to artifact (./outputs-9/checkpoint-2820)... Done. 0.1s
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


TrainOutput(global_step=2820, training_loss=0.27308837857298845, metrics={'train_runtime': 3545.0568, 'train_samples_per_second': 9.534, 'train_steps_per_second': 0.795, 'total_flos': 1.55103275778048e+17, 'train_loss': 0.27308837857298845, 'epoch': 20.0})

In [7]:
len(sft_dataset) // 32 * 10

1190

In [8]:
len(trainer.train_dataset) // 32 * 10

520

# prediction

In [ ]:
# load model from output checkpoints
checkpoint_path = 'outputs-6/checkpoint-1050'
model = AutoModelForCausalLM.from_pretrained(checkpoint_path, device_map='auto')
tokenizer = AutoTokenizer.from_pretrained(checkpoint_path)

In [10]:
import itertools
for data in itertools.islice(sft_transform(data_dir, label=False), 5):
  text = data['text']
  device = "cuda:0"
  inputs = tokenizer(text, return_tensors="pt").to(device)

  outputs = model.generate(**inputs, max_new_tokens=256, do_sample=True, top_k=50, top_p=0.95,
                           pad_token_id= tokenizer.eos_token_id, eos_token_id= tokenizer.eos_token_id)

  print(tokenizer.decode(outputs[0], skip_special_tokens=True))


### Human: Generate 10 text ads for the product with the following information.
description: Immerse yourself in the game with this ultra-wide curved gaming monitor.
  With its 4K resolution and HDR support, you'll experience stunning visuals and smooth
  gameplay. The 144Hz refresh rate and 1ms response time ensure blur-free action.
product_category: Electronics -> Computers & Accessories
product_name: Ultra HD 4K Curved Gaming Monitor
selling_point: 4K resolution, HDR support, 144Hz refresh rate, 1ms response time
target_audience: Gamers, video editors, graphic designers

### Assistant:
Immersive 4K Gaming Immersion
Razor-Sharp 144Hz Refresh Rate
Blazing-Fast 1ms Response Time
HDR for Stunning Visuals
Immerse Yourself in the Game
4K Gaming Immersion
HDR Visuals Unleashed
144Hz Blur-Free Action
1ms Response Time Precision
Curved for Cinematic Gaming
### Human: Generate 10 text ads for the product with the following information.
description: Enjoy your music without the hassle of wires

In [ ]:
df = sample_generation(100)
for sample in df['sample']:
  print(sample)

4096

In [14]:
wandb.finish()

train/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
train/grad_norm,▆▄▅▄▅▅▆▆▆▇▇▇▇█▇▇▆▆▆▅▄▃▂▂▁▂▁▁▂▁▁▁▁▁▁▁▁▁▁▁
train/learning_rate,████▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▁▁▁
train/loss,█▇▆▆▅▅▄▅▄▄▃▃▂▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/total_flos,▁
train/train_loss,▁
train/train_runtime,▁
train/train_samples_per_second,▁
train/train_steps_per_second,▁
train/epoch,20.0
